# beomi/KcELECTRA-base

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11406065593058479576
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2254123828
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7713650353449230658
physical_device_desc: "device: 0, name: GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [2]:
# %run KcELECTRA-base.version.py
# !pip freeze

In [1]:
import re
import torch
import pandas as pd

# deivce 선택
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device:", device)

devcie: cpu


In [4]:
# 엑셀 파일에서 데이터프레임 읽기
df = pd.read_excel(r"C:\Users\GJAISCHOOL\Desktop\X_filter\Algorithm\dataset\sentence_data.xlsx")

# 'Sentence' 칼럼의 값을 문자열로 변환하여 리스트로 저장
str_data = df['Sentence'].astype(str).tolist()

# 'Sentence' 칼럼의 값을 하나의 문자열로 결합
all_sentences = ' '.join(str_data)

print(df.shape)
df.head()

(134197, 2)


,Sentence,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1


In [5]:
null_idx = df[df.label.isnull()].index                             # 해당 index에 null 값 확인
df.loc[null_idx, "Sentence"]                                       # null 값이 존재한 인덱스의 content 값 불러오기

# lable은 content의 가장 끝 문장열로 설정
df.loc[null_idx, "label"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-1])

# content는 "\t" 앞부분까지의 문자열로 설정
df.loc[null_idx, "Sentence"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-2])

In [6]:
train_data = df.sample(frac=0.8, random_state=42)                 # train(80%), test(20%) 셋 구분 
test_data = df.drop(train_data.index)                             # 랜덤으로 샘플링(랜덤으로 숫자 배치)

# 데이터셋 갯수 확인
print('중복 제거 전 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋 : {}'.format(len(test_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=["Sentence"], inplace=True)
test_data.drop_duplicates(subset=["Sentence"], inplace=True)

# 데이터셋 갯수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))

중복 제거 전 학습 데이터셋 : 107358
중복 제거 전 테스트 데이터셋 : 26839
중복 제거 후 학습 데이터셋 : 103690
중복 제거 후 테스트 데이터셋 : 26600


In [7]:
print(type(train_data))
print(type(test_data))
train_data['Sentence']

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


37135     \n세계경제는 일본''놈'''이 ''말아 처먹'''고 한국경제는 #@이름#이 ''말...
14029                                               기안내나노 ㅋ
120928    진영논리 빼곤 아무것도 없는 ''스레기당''' 민주당이 하는''짓'''이라곤 ''양...
17981         개인적으로 기독교 비호감이랰ㅋㅋㅋㅋ지랄한닼ㅋㅋㅋㅋ 오랑캐종교믿는 광신도갘ㅋㅋㅋㅋㅋ
66283     그래~ 북조선 애들보다 차라리 저런애들 걷어주는게 더 낫다. 쟤들은 고마움이라도 느끼지~
                                ...                        
17899                                             난민맞냐씨발ㅋㅋㅋ
111752    정말 열받는 얘기만 쏟아내네!언제부터 부산시에서 문화재신경쓰고 보호했다고!반대하는 ...
35423     ''어처구니가 없'''다!! 니들이 그렇게 싫어하는 일본을 막기 위해서라도 전쟁할수...
14570               아니 근데 막상 시키면 왜이런거 시키냐고 그러고 농땡이부리면서 안하던데
134062    확실이 여자들이 투표하는걸 좋아하네 4년째 이런프로를 ''빨'''아주는거보니 ㅋㅋ ...
Name: Sentence, Length: 103690, dtype: object

In [8]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, TFElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# KcELECTRA 모델 및 토크나이저 로드
model_name = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\GJAISCHOOL\.conda\envs\nv38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
train_data

,Sentence,label
37135,\n세계경제는 일본''놈'''이 ''말아 처먹'''고 한국경제는 #@이름#이 ''말...,1
14029,기안내나노 ㅋ,1
120928,진영논리 빼곤 아무것도 없는 ''스레기당''' 민주당이 하는''짓'''이라곤 ''양...,1
17981,개인적으로 기독교 비호감이랰ㅋㅋㅋㅋ지랄한닼ㅋㅋㅋㅋ 오랑캐종교믿는 광신도갘ㅋㅋㅋㅋㅋ,1
66283,그래~ 북조선 애들보다 차라리 저런애들 걷어주는게 더 낫다. 쟤들은 고마움이라도 느끼지~,1
...,...,...
17899,난민맞냐씨발ㅋㅋㅋ,1
111752,정말 열받는 얘기만 쏟아내네!언제부터 부산시에서 문화재신경쓰고 보호했다고!반대하는 ...,1
35423,''어처구니가 없'''다!! 니들이 그렇게 싫어하는 일본을 막기 위해서라도 전쟁할수...,1
14570,아니 근데 막상 시키면 왜이런거 시키냐고 그러고 농땡이부리면서 안하던데,1


In [10]:
tokenized_train_sentences = tokenizer(
    list(train_data["Sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [11]:
tokenized_test_sentences = tokenizer(
    list(test_data["Sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [12]:
print(tokenized_train_sentences[0])        # 0번째 문장에 해당하는 객체 
print(tokenized_train_sentences[0].tokens) # 0번째 문장에 토큰의 목록
print(tokenized_train_sentences[0].ids)    # 0번째 문장에 대한 고유 ID

Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['Ġ', 'Ċ', 'ìĦ¸ê³Ħ', 'ê²½ìłľëĬĶ', 'ĠìĿ¼ë³¸', "''", 'ëĨĪ', "''", "'", 'ìĿ´', "Ġ''", 'ë§ĲìķĦ', 'Ġì²ĺë¨¹', "''", "'", 'ê³ł', 'ĠíķľêµŃê²½ìłľ', 'ëĬĶ', 'Ġ#', '@', 'ìĿ´ë¦Ħ', '#', 'ìĿ´', "Ġ''", 'ë§ĲìķĦ', 'Ġì²ĺë¨¹', "''", "'", 'ê³ł', 'ĠìŀĪëĭ¤', 'Ġìłķë§Ĳ', 'Ġë¯¸ì¹ĺê²łëĭ¤', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [13]:
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encoding, labels):
        self.encodings = encoding
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여
        item["labels"] = torch.tensor(self.labels[idx])                             # key: toch.tensor(val[ix])라는 item 딕셔너리 형성
        return item                                                                 # 새로운 labels 키 값에 value torch.tensor(self.labels[idx]) 쌍을 생성
    
    def __len__(self):
        return len(self.labels)                                                     # self.labels 길이 반환

In [14]:
# train_set, test_set에 대한 데이터셋을 각각 생성

train_label = train_data["label"].values
test_label = test_data["label"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)   # 사전 학습된 모델 찾아오기
model.to(device)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.ou

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [16]:
# 3 - 2 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir = './results',           # 학습결과 저장경로
    num_train_epochs=10,                # 학습 epoch 설정
    per_device_train_batch_size = 8,    # train batch_size 설정
    per_device_eval_batch_size = 64,    # test batch_size 설정
    logging_dir = './logs',             # 학습 log 저장경로
    logging_steps=500,                  # 학습 log 기록 단위
    save_total_limit = 2,               # 학습결과 저장 최대갯수
)

In [17]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# 학습과정에서 사용할 평가지표를 위한 함수 설정
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # 정밀도, 재현율, f1 구하기 
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
  # 정확도 구하기
  acc = accuracy_score(labels, preds)
  return{
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

In [18]:
# Trainer 모듈을 사용해 모델의 학습을 컨트롤하은 trainer를 생성
trainer = Trainer(
    model = model,                       # 학습하고자하는 Transformers model
    args=training_args,                  # 위에서 정의한 Trainging Arguments
    train_dataset=train_dataset,         # 학습 데이터셋
    eval_dataset=test_dataset,           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 평가지표
)

In [19]:
type(trainer)

transformers.trainer.Trainer

In [20]:
trainer.train()

c:\Users\GJAISCHOOL\.conda\envs\nv38\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/129620 [00:00<?, ?it/s]C:\Users\GJAISCHOOL\AppData\Local\Temp\ipykernel_12096\1167134061.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여
  0%|          | 10/129620 [00:59<197:31:26,  5.49s/it]

KeyboardInterrupt: 

In [ ]:
# 평가를 수행하고 custom_ 접두사가 붙은 평가 지표를 출력
results = trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix='custom_')

print(results)

In [ ]:
# 0: curse, 1: non_curse
def sentence_predict(sent):
    # 평가모드로 변경
    model.eval()

    # 입력된 문장 토크나이징
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128,
    )

    # 모델이 위치한 GPU로 이동
    tokenized_sent.to(deivce)

    # 예측
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"],
        )

    # 결과 return
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    sentence = True
    input_sentence = tokenized_sent["input_ids"]
    if result == 0:
        sentence = True

    elif result == 1:
        sentence = False
    return sentence, input_sentence

In [ ]:
def found_word(input_sentence, found_bad_word):
    result = input_sentence
    if found_bad_word:
        result = badword_find(result)
    return result

def badword_find(input_sentence):
    result = input_sentence
    badword_df = pd.read_excel(r'C:\Users\user\Desktop\X-filter\word_list.xlsx')
    
    found_bad_word = False  # 입력 문장에 단어가 발견되었는지를 나타내는 플래그
    for idx, row in badword_df.iterrows():
        if row["WORD"] in input_sentence:
            # 'WORD'가 입력 문장에 포함된 경우
            new_word = row["대체어"]
            if not pd.isnull(new_word):
                result = result.replace(row["WORD"], new_word)
                found_bad_word = True
            else:
                result = result.replace(row["WORD"], "*" * len(row["WORD"]))
                found_bad_word = True
    
    if not found_bad_word:
        # result = "@" * len(input_sentence)
        result = "혐오 표현입니다."
    return result

# 테스트
input_sentence = "엄마가 죽었으면 좋겠어"
speak_result = badword_find(input_sentence)
print(speak_result)

In [ ]:
badword_find(input_sentence)

In [ ]:
def speak(sent):
    sentence = sentence_predict(sent)
    if sentence[0] == False:
        return sent
    elif sentence[0] == True:
        return badword_find(sent)

In [ ]:
import kss
from pykospacing import Spacing
from soynlp.normalizer import repeat_normalize

# 띄어쓰기 설정
spacing = Spacing()

In [ ]:
# 특수문자 제거
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ|a-zA-Z0-9]','', text)
    return text

In [ ]:
def clean_and_repeat_normalize(text):
    cleansed_text = cleanse(text)                                             # 특수문자 제거
    normalized_text = repeat_normalize(cleansed_text, num_repeats=2)          # 중복문자 제거
    input_data = re.sub(r'\d', '', normalized_text)                           #  제거
    normalized_text = spacing(input_data)                                     # 띄어쓰기 보정 

    return normalized_text

clean_and_repeat_normalize("아버지가방에들어가신다")

In [ ]:
def final_output():
    input_text = clean_and_repeat_normalize(input())
    sentences = kss.split_sentences(input_text)

    sentences_list = []
    for sentence in sentences:
        text_output = speak(sentence)
        sentences_list.append(text_output)

    long_test = ' '.join(sentences_list)
    print(long_test)
    return long_test

In [ ]:
final_output()